In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/Face_Recognition

/content/drive/My Drive/Colab Notebooks/Face_Recognition


In [3]:
path='/content/drive/My Drive/Colab Notebooks/Face_Recognition'

In [ ]:
! pip install gdown
! pip install tensorflow==2.0.0

!pip install twilio
!pip install redmail

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.0.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.13.0rc0, 2.13.0rc1)
ERROR: No matching distribution found for tensorflow==2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Libraries used

# Usual Libraries
import os
import re
import glob
from google.colab.patches import cv2_imshow
from datetime import datetime

# For Face Detection
import cv2
import matplotlib.pyplot as plt
import dlib

# Tensorflow Library Mainly for keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

# Threading for making the testing faster
import threading
from collections import defaultdict

import pandas as pd

# For Sending the attendance Information through Mass Media like Whatsapp and Gmail.

#For Whatsapp
from twilio.rest import Client

#For Gmail
from redmail import outlook


In [ ]:
# Get Image names stored in "Images" folder
image_path_names=[]
person_names=set()
for file_name in glob.glob(path+'/Images/*[1-9]*.jpg'):
  image_path_names.append(file_name)
  person_names.add(re.sub(r'[0-9]', '', image_path_names[-1].split('/')[-1].split('.')[0]))

In [ ]:
#Total Number of Student Name
print("Total Number of students faces are {0}".format(len(image_path_names)))

Total Number of students faces are 78


In [ ]:
# All the names of Students
print("All Student names",person_names)

All Student names {'Ranjith', 'NTR', 'RamCharanTeja', 'Pragna', 'MalliReddy', 'Pragna ()', 'saiCharanTeja', 'Shiva'}


  There are total 70 images containing 10 images per person.

In [ ]:
# Download Dlib CNN face detector
! wget http://dlib.net/files/mmod_human_face_detector.dat.bz2

--2023-06-13 12:57:20--  http://dlib.net/files/mmod_human_face_detector.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694709 (678K)
Saving to: ‘mmod_human_face_detector.dat.bz2’

mmod_human_face_det 100%[===================>] 678.43K  --.-KB/s    in 0.09s   

2023-06-13 12:57:20 (6.99 MB/s) - ‘mmod_human_face_detector.dat.bz2’ saved [694709/694709]



In [ ]:
!bzip2 -dk mmod_human_face_detector.dat.bz2

bzip2: Output file mmod_human_face_detector.dat already exists.


In [ ]:
%rm mmod_human_face_detector.dat.bz2

In [ ]:
# Load CNN face detector into dlib
dnnFaceDetector=dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

RuntimeError: ignored

In [ ]:
# Create a Folder Image_Crop
if(os.path.exists(path+'/Images_crop/')== False):
    os.mkdir(path+'/Images_crop/')

In [ ]:
# For each person create a separate folder
for person in person_names:
    if(os.path.exists(path+'/Images_crop/'+person+'/') == False):
        os.mkdir(path+'/Images_crop/'+person+'/')

In [ ]:
def Crop_Detected_Faces(file_name,img_path):
    img=cv2.imread(file_name)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    rects=dnnFaceDetector(gray,1)

    left,top,right,bottom=0,0,0,0
    for (i,rect) in enumerate(rects):
        left=rect.rect.left() #x1
        top=rect.rect.top() #y1
        right=rect.rect.right() #x2
        bottom=rect.rect.bottom() #y2
    width=right-left
    height=bottom-top
    img_crop=img[abs(top):abs(top)+height,abs(left):abs(left)+width]

    cv2.imwrite(img_path,img_crop)


In [ ]:
# Detect face, crop detected face and save them in corresponding person folder
threads = []
for file_name in image_path_names:
    img_path=path+'/Images_crop/'+re.sub(r'[0-9]', '', file_name.split('/')[-1].split('.')[0])+"/"+file_name.split('/')[-1]
    t = threading.Thread(target=Crop_Detected_Faces, args=(file_name,img_path,))

    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

In [ ]:
# Get Image names for testing
test_image_path_names=[]
for file_name in glob.glob(path+'/Images_test/*_[123].jpg'):
  test_image_path_names.append(file_name)

In [ ]:
print("Total number of student test images {0}".format(len(test_image_path_names)))

Total number of student test images 21


For each person 3 images to test in Images_test folder.

In [ ]:
# Creating a Test Images Crop folder
if(os.path.exists(path+'/Test_Images_crop/') == False):
    os.mkdir(path+'/Test_Images_crop/')

In [ ]:
# Create Separate folder for each person in "Test_Images_crop" folder
for person in person_names:
    if(os.path.exists(path+'/Test_Images_crop/'+person+'/') == False):
        os.mkdir(path+'/Test_Images_crop/'+person+'/')

In [ ]:
# Detect face,crop face and save in corresponding folder
threads = []
for file_name in test_image_path_names:
    img_path=path+'/Test_Images_crop/'+file_name.split('/')[-1].split('_')[0]+'/'+file_name.split('/')[-1]
    t = threading.Thread(target=Crop_Detected_Faces, args=(file_name,img_path))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

<pre>Here images are sorted to corresponding test and train folders of same person
Directory structure :
|Images /
|  |-- (60 images) - Images of student. 10 Images of each student.
|Images_crop /
|  |--SaiCharanTeja /
|     |--(10 images)
|  |--Pargna /
|     |--(10 images)
|  |--MalliReddy /
|         |--(10 imgaes)
|  |--Pranay /
|         |--(10 imgaes)
|Images_test / - Given Images to test the model.Each of 3 Images
|  |-- .. / (18 images)
|Images_test_crop /
|  |--SaiCharanTeja / (3 images)
|  |--Pargna / (3 images)
|  |--MalliReddy / (3 images)
|  |--Pranay / (3 images)
|Test_Images / - Images  to be tested.
|  |-- There is no specified Number of Images.
|  |--Predictions / - The predicted Images are stored with in this folder.
|  |--Attendance/ - To store the daily attendance
|vgg_face_weights.h5
|train_data.npy
|train_labels.npy
|test_data.npy
|test_labels.npy
|Face_Recognition.ipynb
|mmod_human_face_detector.dat
|face_classifier_model.h5
</pre>

In [ ]:
%ls

face_classifier_model.h5      StudentAttendance/  test_labels.npy
Images/                       Students_Info.xlsx  train_data.npy
Images_crop/                  test_data.npy       train_labels.npy
Images_test/                  Test_Images/        vgg-face-keras-fc.h5
mmod_human_face_detector.dat  Test_Images_crop/   vgg_face_weights.h5
Predictions/                  Test_Images_Crop/


In [ ]:
#Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
# Load VGG Face model weights
model.load_weights(path+'/vgg_face_weights.h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                        

In [ ]:
# Remove Last Softmax layer and get model upto last flatten layer with outputs 2622 units
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)

In [ ]:
#Prepare Training Data
x_train=[]
y_train=[]
person_folders=os.listdir(path+'/Images_crop/')

person_rep=dict()
for i,person in enumerate(person_folders):
  person_rep[i]=person
  image_names=os.listdir('Images_crop/'+person+'/')
  for image_name in image_names:
    img=load_img(path+'/Images_crop/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    img_encode=vgg_face(img)
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
    y_train.append(i)

In [ ]:
person_rep

{0: 'RamCharanTeja',
 1: 'MalliReddy',
 2: 'Ranjith',
 3: 'Pragna ()',
 4: 'NTR',
 5: 'saiCharanTeja',
 6: 'Pragna',
 7: 'Shiva'}

In [ ]:
x_train=np.array(x_train)
y_train=np.array(y_train)

In [ ]:
#Prepare Test Data
x_test=[]
y_test=[]
person_folders=os.listdir(path+'/Test_Images_crop/')
for i,person in enumerate(person_folders):
  image_names=os.listdir('Test_Images_crop/'+person+'/')
  for image_name in image_names:
    img=load_img(path+'/Test_Images_crop/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)

    img=preprocess_input(img)
    img_encode=vgg_face(img)

    x_test.append(np.squeeze(K.eval(img_encode)).tolist())
    y_test.append(i)

In [ ]:
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
# Save test and train data for later use
np.save('train_data',x_train)
np.save('train_labels',y_train)
np.save('test_data',x_test)
np.save('test_labels',y_test)

In [ ]:
# Load saved data
x_train=np.load('train_data.npy')
y_train=np.load('train_labels.npy')
x_test=np.load('test_data.npy')
y_test=np.load('test_labels.npy')

In [ ]:
# Softmax regressor to classify images based on encoding
classifier_model=Sequential()

classifier_model.add(Dense(units=100,input_dim=x_train.shape[1],kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.3))

classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.2))

classifier_model.add(Dense(units=8,kernel_initializer='he_uniform'))
classifier_model.add(Activation('softmax'))
# classifier_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),optimizer='nadam',metrics=['accuracy'])
#classifier_model.compile(loss=tf.keras.losses.binary_crossentropy(),optimizer='Adam',metrics=['accuracy'])
classifier_model.compile(loss="sparse_categorical_crossentropy",optimizer='Nadam',metrics=['accuracy'])

In [ ]:
classifier_model.fit(x_train,y_train,epochs=250,validation_data=(x_test,y_test))

Epoch 1/250
3/3 [==============================] - 4s 135ms/step - loss: 1.7823 - accuracy: 0.3462 - val_loss: 1.3095 - val_accuracy: 0.6667
Epoch 2/250
3/3 [==============================] - 0s 20ms/step - loss: 1.1224 - accuracy: 0.7051 - val_loss: 1.0803 - val_accuracy: 0.7619
Epoch 3/250
3/3 [==============================] - 0s 18ms/step - loss: 0.8328 - accuracy: 0.8846 - val_loss: 1.1026 - val_accuracy: 0.7143
Epoch 4/250
3/3 [==============================] - 0s 19ms/step - loss: 0.7574 - accuracy: 0.9231 - val_loss: 1.2065 - val_accuracy: 0.6190
Epoch 5/250
3/3 [==============================] - 0s 19ms/step - loss: 0.7217 - accuracy: 0.8846 - val_loss: 1.1375 - val_accuracy: 0.6667
Epoch 6/250
3/3 [==============================] - 0s 19ms/step - loss: 0.7289 - accuracy: 0.9359 - val_loss: 1.0517 - val_accuracy: 0.6667
Epoch 7/250
3/3 [==============================] - 0s 20ms/step - loss: 0.6425 - accuracy: 0.9872 - val_loss: 1.0361 - val_accuracy: 0.7143
Epoch 8/250
3/3 [==

In [ ]:
# Save model for later use
tf.keras.models.save_model(classifier_model,'/content/drive/My Drive/Colab Notebooks/Face_Recognition/face_classifier_model.h5')

In [ ]:
# Load saved model
classifier_model=tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/Face_Recognition/face_classifier_model.h5')

In [ ]:
# Path to folder which contains images to be tested and predicted
test_images_path=path+'/Test_Images/'

In [ ]:
dnnFaceDetector=dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

In [ ]:
def plot(img):
  plt.figure(figsize=(8,4))
  plt.imshow(img[:,:,::-1])
  plt.show()

In [ ]:
from zmq.sugar import frame
video_path = "/content/drive/MyDrive/videos/Natu_Natu.mp4"
output_path = "/content/drive/MyDrive/Colab Notebooks/Face_Recognition/Test_Images"


vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()
count = 0
frameNum =0
while success:
    if count % 1000 == 0:
        cv2.imwrite(output_path + "/frame%d.jpg" %frameNum , image)
        frameNum += 1
    success, image = vidcap.read()
    count += 20


In [ ]:
# Label names for class numbers
# person_rep={0: 'Ranjith', 1: 'Shiva', 2: 'saiCharanTeja',3:"None"}

In [ ]:
if(os.path.exists(path+'/Predictions')==False):
    os.mkdir(path+'/Predictions')

In [ ]:
attendance = dict();
image_count =0
for img_name in os.listdir('Test_Images/'):
  try:
    if img_name=='crop_img.jpg':
        continue
    # Load Image
    img=cv2.imread(path+'/Test_Images/'+img_name)
    print(path+'/Test_Images/'+img_name)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image_count+=1
    # Detect Faces
    rects=dnnFaceDetector(gray,1)
    left,top,right,bottom=0,0,0,0
    for (i,rect) in enumerate(rects):
        # Extract Each Face
        left=rect.rect.left() #x1
        top=rect.rect.top() #y1
        right=rect.rect.right() #x2
        bottom=rect.rect.bottom() #y2
        width=right-left
        height=bottom-top
        img_crop=img[abs(top):abs(top)+height,abs(left):abs(left)+width]
        cv2.imwrite(path+'/Test_Images/crop_img.jpg',img_crop)

        # Get Embeddings
        crop_img=load_img(path+'/Test_Images/crop_img.jpg',target_size=(224,224))
        crop_img=img_to_array(crop_img)
        crop_img=np.expand_dims(crop_img,axis=0)
        crop_img=preprocess_input(crop_img)
        img_encode=vgg_face(crop_img)

        # Make Predictions
        embed=K.eval(img_encode)
        person=classifier_model.predict(embed)
        name=person_rep[np.argmax(person)]
        os.remove(path+'/Test_Images/crop_img.jpg')

        cv2.rectangle(img,(abs(left),abs(top)),(right,bottom),(0,255,0), 2)
        if(np.max(person)>0.75):
            img=cv2.putText(img,name,(abs(left),abs(top)-10),cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,255),2,cv2.LINE_AA)
            img=cv2.putText(img,str(np.max(person)),(right,bottom+10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)
            if(name in attendance.keys()):
                attendance[name]+=1;
            else:
                attendance[name] =0;
            print(str(np.max(person)),"==>",name)

    # Save images with bounding box,name and accuracy
    cv2.imwrite(path+'/Predictions/'+img_name,img)
    plot(img)
  except:
    pass


/content/drive/My Drive/Colab Notebooks/Face_Recognition/Test_Images/Copy of Sai (1).jpg
1/1 [==============================] - 0s 18ms/step
/content/drive/My Drive/Colab Notebooks/Face_Recognition/Test_Images/Copy of MalliT3.jpeg
1/1 [==============================] - 0s 21ms/step
0.97336864 ==> MalliReddy
/content/drive/My Drive/Colab Notebooks/Face_Recognition/Test_Images/Copy of Shivvvvsaii.jpg
1/1 [==============================] - 0s 29ms/step
0.95217246 ==> saiCharanTeja
1/1 [==============================] - 0s 27ms/step
0.92460006 ==> Shiva
/content/drive/My Drive/Colab Notebooks/Face_Recognition/Test_Images/Copy of Shovsai.jpg
1/1 [==============================] - 0s 20ms/step
0.91867775 ==> saiCharanTeja
1/1 [==============================] - 0s 20ms/step
0.76289934 ==> Shiva
/content/drive/My Drive/Colab Notebooks/Face_Recognition/Test_Images/Copy of Saii.jpg
1/1 [==============================] - 0s 20ms/step
0.9149128 ==> saiCharanTeja
/content/drive/My Drive/Colab Noteb

In [ ]:
# Function to predict faces in a name and update the frequency dictionary

def process_name(img_name, frequency_dict):
  try:
    img=cv2.imread(path+'/Test_Images/'+img_name)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect Faces
    rects=dnnFaceDetector(gray,1)
    left,top,right,bottom=0,0,0,0
    for (i,rect) in enumerate(rects):
        # Extract Each Face
        left=rect.rect.left() #x1
        top=rect.rect.top() #y1
        right=rect.rect.right() #x2
        bottom=rect.rect.bottom() #y2
        width=right-left
        height=bottom-top
        img_crop=img[abs(top):abs(top)+height,abs(left):abs(left)+width]
        cv2.imwrite(path+'/Test_Images_Crop/'+img_name[:-4]+'crop_img.jpg',img_crop)
        # Get Embeddings
        crop_img=load_img(path+'/Test_Images_Crop/'+img_name[:-4]+'crop_img.jpg',target_size=(224,224))
        crop_img=img_to_array(crop_img)
        crop_img=np.expand_dims(crop_img,axis=0)
        crop_img=preprocess_input(crop_img)
        img_encode=vgg_face(crop_img)
        # Make Predictions
        embed=K.eval(img_encode)
        person=classifier_model.predict(embed)
        name=person_rep[np.argmax(person)]
        os.remove(path+'/Test_Images_Crop/'+img_name[:-4]+'crop_img.jpg')
        cv2.rectangle(img,(abs(left),abs(top)),(right,bottom),(0,255,0), 2)
        if(np.max(person)>0.75):
            img=cv2.putText(img,name,(abs(left),abs(top)-10),cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,255),2,cv2.LINE_AA)
            img=cv2.putText(img,str(np.max(person)),(right,bottom+10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)
            # Update the frequency dictionary
            frequency_dict[name] +=1

    # Save images with bounding box,name and accuracy
    cv2.imwrite(path+'/Predictions/'+img_name,img)
  except:
    pass


In [ ]:
# List of names to process
names = list(person_names)

# Dictionary to store the frequency of faces in each name
frequency_dict = defaultdict(int)

for i in names:
    frequency_dict[i] =0

# Create a lock to synchronize access to the dictionary
lock = threading.Lock()

# Create threads to process each name
threads = []
for img_name in os.listdir('Test_Images/'):
    if img_name=='crop_img.jpg':
        continue
    t = threading.Thread(target=process_name, args=(img_name, frequency_dict))

    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

# Print the frequencies of faces in each name
for name, frequency in frequency_dict.items():
    print(f"Name: {name}, Frequency: {frequency}")
attendance = frequency_dict

1/1 [==============================] - 0s 68ms/step
Name: NTR, Frequency: 11
Name: Ranjith, Frequency: 0
Name: Pragna (), Frequency: 36
Name: Shiva, Frequency: 2
Name: MalliReddy, Frequency: 12
Name: Pragna, Frequency: 0
Name: saiCharanTeja, Frequency: 6
Name: RamCharanTeja, Frequency: 15


In [ ]:
class Student:
    def __init__(self, name, MobileNumber,Mail):
        self.name = name
        self.MobileNumber = MobileNumber
        self.Mail = Mail

In [ ]:
#load Student Info details
df = pd.read_excel(path+'/Students_Info.xlsx',index_col=0);
students_Info =[]
for idx in df.index:
    students_Info.append(Student(df["Name"][idx],df["Mobile_Number"][idx],df["Mail_Id"][idx]))

In [ ]:
for i in students_Info:
    print(i.name,i.MobileNumber,i.Mail)

Ranjith 919346305093 sadamharshavardhan@gmail.com
Shiva 919640289787 shivakumardarmod@gmail.com
saiCharanTeja 919390919744 saicharantejardss3661@gmail.com
Pragna 918074845785 somabathulapragnasri@gmail.com
MalliReddy 918919058318 nagamalleshwarivemireddy@gmail.com
Pragna () 918074845785 somabathulapragnasri@gmail.com
NTR 919390919744 saicharantejardss3661@gmail.com
RamCharanTeja 919390919744 saicharantejardss3661@gmail.com


In [ ]:
if(os.path.exists(path+'/StudentAttendance/')== False):
    os.mkdir(path+'/StudentAttendance/')

In [ ]:
# datetime object containing current date and time
now = datetime.now()

# dd/mm/YY H:M:S
dt_string = now.strftime("D_%d_%m_%Y_T_%H_%M_%S.xlsx")

dict_ ={"Name":attendance.keys(),"Attentiveness":attendance.values()}

df = pd.DataFrame(dict_)
df.to_excel(path+'/StudentAttendance/'+dt_string)
df = pd.read_excel(path+'/StudentAttendance/'+dt_string,index_col=0)
df

,Name,Attentiveness
0,NTR,11
1,Ranjith,0
2,Pragna (),36
3,Shiva,2
4,MalliReddy,12
5,Pragna,0
6,saiCharanTeja,6
7,RamCharanTeja,15


In [ ]:
def SendMessage(message,mobileNumber):
    account_sid = "AC601b4a38f1490c4784ff100b3d906d03"
    auth_token = "9e6485e9f8c4f50fa48287f42966214a"

    client = Client(account_sid, auth_token)
    message = client.messages.create(
    body=message,
    from_="whatsapp:+14155238886",
    to="whatsapp:+"+str(mobileNumber)
    )

In [ ]:
for i in students_Info:
    attentiveness = 0
    if(i.name in attendance.keys()):
        attentiveness = attendance[i.name]

    print("Message Successfully sent to :" +i.name)
    msg ="Hello : "+i.name+ ",\n Your attentiveness Percentage is "+str(attentiveness)+" %"
    if(attentiveness <50):
         msg = msg + "\n Make sure to Focus on the class Next time."
         msg = msg + ".\nAnd You are Absent for the class Today\n"
    else:
         msg = msg + ". \nAnd You are Present for the class Today.\n"
    msg+="Regards JNTUHCES BATCH 7"
    SendMessage(msg,i.MobileNumber)


In [ ]:
outlook.username = "projectkxkx@outlook.com"
outlook.password = "Shubhi@#Shubhi"

def sendmail(candidate_email,attentive,attentiveness):
    status = "PRESENT"
    header = "This is "
    footer ="Your active participation and attentiveness were commendable, and we appreciate your dedication to the class.</p><p>Keep up the excellent work!</p>"
    if attentive == False:
        status = "ABSENT"
        header = 'We regret '
        footer =" We kindly request that you make a conscious effort to be more attentive during future classes.</p><p>Keep up the excellent work!</p>"

    message = '''
        <!DOCTYPE html>
<head>
<style>
    body {
        background-image: url('https://jntuh.ac.in/images/jntuhlogo.png');
        background-size: cover;
        background-repeat: no-repeat;
        background-color: rgba(20, 136, 234, 0.2);
        opacity:1;
        z-index: -1;
    }

    #card {
        position: relative;
        top:60px;
        width: 320px;
        display: block;
        margin: auto;
        text-align: center;
        font-family: 'Source Sans Pro', sans-serif;
        border: 6px solid  #0000FF;
        border-radius: 10px;


    }

    #upper-side {
        padding: 2em;
        background-color: rgba(0, 0, 255, 1);
        display: block;
        color: #fff;

        border-bottom-right-radius: 10px;
        border-bottom-left-radius: 10px;

    }


    #status {
        font-weight: lighter;
        text-transform: uppercase;
        letter-spacing: 5px;
        font-size: 1em;
        margin-top: -0.2em;
        margin-bottom: 0;
    }

    #lower-side {
        padding: 2em 2em 5em 2em;
        background: #fff;
        display: block;
        border-bottom-right-radius: 10px;
        border-bottom-left-radius: 10px;
    }

    #message {
        margin-top: -0.5em;
        color: #757575;
        letter-spacing: 1px;
        color: #000000;
  		font-size: 16px;
  		font-weight: bold;
  		font-style: normal;

    }
.envelope-header {
      margin-bottom: 30px;
       text-align: center;
    }

</style>

</head>

<html>
<body>
<div class="envelope-header">
      <img src="https://jntuh.ac.in/images/jntuhlogo.png" alt="Attendance">
    </div>
<div id="card" class="animated fadeIn">

    <div id="upper-side">
        <h1 id="status"><b>Attendance Status</b></h1>
    </div>
    <div id="lower-side">
        <p id="message">'''+ header +'''to inform you that you were marked <h3> <b style="color: #FF0000;">'''+ status+''' </b> </h3>
        </p>
        <p id="message">'''+footer+'''
        <p style="color: #006400;font-weight: bold; "><b>And Your percentage of Attentativeness is</b>
        	<h1>
              	<b style="color: #00FF00;">'''+ str(attentiveness) +'''%
            	<b>
             </h1>
        </p>
  	</div>
</div>

</body>
</html>

'''
    outlook.send(
        receivers=[candidate_email,],
        subject="Your Attendance @ JNTU",
        html=message
    )

In [ ]:
#Mailing
for i in students_Info:
    status = False
    attentiveness = 0
    if(i.name in attendance.keys()):
        if(attendance[i.name] > 50):
            status = True
        attentiveness = attendance[i.name]
    sendmail(i.Mail,status,attentiveness)